# Feature Engineering

In [4]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install tldextract python-whois

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: tldextract in c:\python312\lib\site-packages (5.1.2)



DEPRECATION: Loading egg at c:\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
import pandas as pd
import re
import tldextract
import requests
import os
from urllib.parse import urlparse
import socket
from dotenv import load_dotenv
from tqdm import tqdm
import time

# === Load API Keys ===
load_dotenv()
WHOIS_API_KEY = os.getenv("WHOIS_API_KEY")  # <-- Make sure this matches your .env file!

# === Paths ===
INPUT_FILE = r"C:\prateek\projects\linkload\backend\ml_models\phishing_detection\data\labeled_url_dataset.csv"
OUTPUT_FILE = r"C:\prateek\projects\linkload\backend\ml_models\phishing_detection\data\phishing_dataset_with_features.csv"

# === WHOIS Helper with Retry & Rate Limit Handling ===
def is_domain_registered(domain, max_retries=3):
    for attempt in range(max_retries):
        try:
            url = (
                f"https://www.whoisxmlapi.com/whoisserver/WhoisService"
                f"?apiKey={WHOIS_API_KEY}&domainName={domain}&outputFormat=JSON"
            )
            response = requests.get(url, timeout=30)
            if response.status_code == 429:  # Too Many Requests
                print(f"Rate limited, waiting to retry {domain}...")
                time.sleep(1)
                continue
            data = response.json()
            if "WhoisRecord" in data and data["WhoisRecord"].get("createdDate"):
                return 1
            return 0
        except Exception as e:
            print(f"WHOIS API error for {domain} (attempt {attempt+1}): {e}")
            if attempt < max_retries - 1:
                time.sleep(5)
    return 0

# === Feature Extraction ===
def extract_features(url):
    features = {}
    try:
        parsed = urlparse(url)
        hostname = parsed.hostname or ""
        domain_data = tldextract.extract(url)
        domain = domain_data.domain + '.' + domain_data.suffix

        # URL-based features
        features["url_length"] = len(url)
        features["hostname_length"] = len(hostname)
        features["path_length"] = len(parsed.path)
        features["num_dots"] = url.count(".")
        features["num_hyphens"] = url.count("-")
        features["has_at_symbol"] = "@" in url
        features["has_double_slash"] = '//' in url[8:]
        features["has_ip_address"] = 1 if re.match(r"(?:\d{1,3}\.){3}\d{1,3}", hostname) else 0
        features["has_https"] = 1 if parsed.scheme == "https" else 0
        features["count_www"] = url.count("www")
        features["count_subdomain"] = len(domain_data.subdomain.split(".")) if domain_data.subdomain else 0
        features["first_dir_length"] = len(parsed.path.split("/")[1]) if len(parsed.path.split("/")) > 1 else 0
        features["has_php_or_html"] = int(".php" in url or ".html" in url or ".htm" in url)

        # WHOIS-based feature
        features["whois_registered"] = is_domain_registered(domain)

        # DNS-based feature
        try:
            socket.gethostbyname(hostname)
            features["dns_resolves"] = 1
        except:
            features["dns_resolves"] = 0

    except Exception as e:
        print(f"[ERROR] Failed to extract from: {url} | {e}")
        return None

    return features

# === Main Processing ===
if __name__ == "__main__":
    df = pd.read_csv(INPUT_FILE)
    df = df.dropna(subset=["url", "label"])

    feature_rows = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        feats = extract_features(row["url"])
        if feats:
            feats["url"] = row["url"]
            feats["label"] = row["label"]
            feature_rows.append(feats)
        time.sleep(0.1)  # Throttle requests to stay well under rate limits

    feature_df = pd.DataFrame(feature_rows)
    feature_df.to_csv(OUTPUT_FILE, index=False)
    print(f"Features extracted and saved to: {OUTPUT_FILE}")


  4%|▍         | 4326/105062 [2:33:45<7250:36:41, 259.11s/it] 

WHOIS API error for roblox.tg (attempt 1): ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))


  4%|▍         | 4683/105062 [2:44:51<57:09:29,  2.05s/it]   

WHOIS API error for bedroomsandmore.com (attempt 1): HTTPSConnectionPool(host='www.whoisxmlapi.com', port=443): Read timed out. (read timeout=30)


  7%|▋         | 7788/105062 [4:19:09<43:57:57,  1.63s/it] 

...

WHOIS API error for navoco.de (attempt 3): HTTPSConnectionPool(host='www.whoisxmlapi.com', port=443): Max retries exceeded with url: /whoisserver/WhoisService?apiKey=32850693e79c4e1291e441e8f4d09707&domainName=navoco.de&outputFormat=JSON (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000028DDCEEE180>: Failed to establish a new connection: [WinError 10051] A socket operation was attempted to an unreachable network'))
WHOIS API error for mywebselfsite.net (attempt 1): HTTPSConnectionPool(host='www.whoisxmlapi.com', port=443): Max retries exceeded with url: /whoisserver/WhoisService?apiKey=32850693e79c4e1291e441e8f4d09707&domainName=mywebselfsite.net&outputFormat=JSON (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000028DDCEEE3C0>: Failed to establish a new connection: [WinError 10051] A socket operation was attempted to an unreachable network'))


 18%|█▊        | 19361/105062 [22:03:02<49:59:30,  2.10s/it]     

WHOIS API error for wixsite.com (attempt 1): HTTPSConnectionPool(host='www.whoisxmlapi.com', port=443): Read timed out. (read timeout=30)


 19%|█▉        | 20267/105062 [22:48:44<95:26:41,  4.05s/it]   


KeyboardInterrupt: 

In [10]:
pip install pandas tldextract requests python-dotenv tqdm


DEPRECATION: Loading egg at c:\python312\lib\site-packages\vboxapi-1.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
import pandas as pd
import re
import tldextract
import requests
import os
from urllib.parse import urlparse
import socket
from dotenv import load_dotenv
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict

# === Load API Keys ===
load_dotenv()
WHOIS_API_KEY = os.getenv("WHOIS_API_KEY")

# === Paths ===
INPUT_FILE = r"C:\prateek\projects\linkload\backend\ml_models\phishing_detection\data\labeled_url_dataset.csv"
OUTPUT_FILE = r"C:\prateek\projects\linkload\backend\ml_models\phishing_detection\data\phishing_dataset_with_features.csv"

# === Global Cache for WHOIS Results ===
whois_cache = {}
dns_cache = {}

# === Configuration ===
SKIP_WHOIS = False  # Set to True to skip WHOIS API calls for faster testing
SKIP_DNS = False    # Set to True to skip DNS lookups for faster testing
MAX_WORKERS = 10    # Number of concurrent threads for WHOIS API calls
BATCH_SIZE = 50     # Number of domains to process in each batch

# === WHOIS Helper with Retry & Rate Limit Handling ===
def is_domain_registered(domain, max_retries=3):
    # Check cache first
    if domain in whois_cache:
        return whois_cache[domain]
    
    for attempt in range(max_retries):
        try:
            url = (
                f"https://www.whoisxmlapi.com/whoisserver/WhoisService"
                f"?apiKey={WHOIS_API_KEY}&domainName={domain}&outputFormat=JSON"
            )
            response = requests.get(url, timeout=30)
            
            if response.status_code == 429:  # Too Many Requests
                print(f"Rate limited, waiting to retry {domain}...")
                time.sleep(1)
                continue
                
            data = response.json()
            result = 1 if "WhoisRecord" in data and data["WhoisRecord"].get("createdDate") else 0
            
            # Cache the result
            whois_cache[domain] = result
            return result
            
        except Exception as e:
            print(f"WHOIS API error for {domain} (attempt {attempt+1}): {e}")
            if attempt < max_retries - 1:
                time.sleep(5)
    
    # Cache failed result as 0
    whois_cache[domain] = 0
    return 0

# === DNS Helper with Caching ===
def check_dns_resolves(hostname):
    # Check cache first
    if hostname in dns_cache:
        return dns_cache[hostname]
    
    try:
        socket.gethostbyname(hostname)
        dns_cache[hostname] = 1
        return 1
    except:
        dns_cache[hostname] = 0
        return 0

# === Feature Extraction (Fast Operations Only) ===
def extract_fast_features(url):
    features = {}
    try:
        parsed = urlparse(url)
        hostname = parsed.hostname or ""
        domain_data = tldextract.extract(url)
        domain = domain_data.domain + '.' + domain_data.suffix

        # URL-based features (fast operations)
        features["url_length"] = len(url)
        features["hostname_length"] = len(hostname)
        features["path_length"] = len(parsed.path)
        features["num_dots"] = url.count(".")
        features["num_hyphens"] = url.count("-")
        features["has_at_symbol"] = "@" in url
        features["has_double_slash"] = '//' in url[8:]
        features["has_ip_address"] = 1 if re.match(r"(?:\d{1,3}\.){3}\d{1,3}", hostname) else 0
        features["has_https"] = 1 if parsed.scheme == "https" else 0
        features["count_www"] = url.count("www")
        features["count_subdomain"] = len(domain_data.subdomain.split(".")) if domain_data.subdomain else 0
        features["first_dir_length"] = len(parsed.path.split("/")[1]) if len(parsed.path.split("/")) > 1 else 0
        features["has_php_or_html"] = int(".php" in url or ".html" in url or ".htm" in url)

        # Store for later processing
        features["hostname"] = hostname
        features["domain"] = domain
        features["url"] = url

        # DNS-based feature (if not skipped)
        if not SKIP_DNS:
            features["dns_resolves"] = check_dns_resolves(hostname)
        else:
            features["dns_resolves"] = 1  # Default value

    except Exception as e:
        print(f"[ERROR] Failed to extract fast features from: {url} | {e}")
        return None

    return features

# === Batch WHOIS Processing ===
def process_whois_for_domains(unique_domains):
    """Process WHOIS for a list of unique domains"""
    results = {}
    
    for domain in tqdm(unique_domains, desc="WHOIS lookups"):
        if not SKIP_WHOIS:
            results[domain] = is_domain_registered(domain)
        else:
            results[domain] = 0  # Default value
        
        # Small delay to respect rate limits
        time.sleep(0.02)  # 50 requests per second max
    
    return results

# === Domain Deduplication ===
def deduplicate_by_domain(df):
    """Group URLs by domain to minimize API calls"""
    print("Analyzing domain distribution...")
    
    # Extract domains for all URLs
    df['extracted_domain'] = df['url'].apply(lambda x: tldextract.extract(x).registered_domain)
    
    # Count URLs per domain
    domain_counts = df['extracted_domain'].value_counts()
    print(f"Total unique domains: {len(domain_counts)}")
    print(f"Total URLs: {len(df)}")
    print(f"Average URLs per domain: {len(df) / len(domain_counts):.2f}")
    
    return df

# === Threaded WHOIS Processing ===
def process_whois_batch_threaded(domain_batch):
    """Process a batch of domains with WHOIS API calls"""
    results = {}
    for domain in domain_batch:
        if not SKIP_WHOIS:
            results[domain] = is_domain_registered(domain)
        else:
            results[domain] = 0
        time.sleep(0.02)  # Rate limiting
    return results

# === Main Processing ===
if __name__ == "__main__":
    print("Loading dataset...")
    df = pd.read_csv(INPUT_FILE)
    df = df.dropna(subset=["url", "label"])
    print(f"Loaded {len(df)} URLs")

    # Deduplicate analysis
    df = deduplicate_by_domain(df)

    # Step 1: Extract fast features for all URLs
    print("\nStep 1: Extracting fast features...")
    feature_rows = []
    
    for _, row in tqdm(df.iterrows(), total=len(df), desc="Fast features"):
        feats = extract_fast_features(row["url"])
        if feats:
            feats["label"] = row["label"]
            feature_rows.append(feats)

    print(f"Extracted fast features for {len(feature_rows)} URLs")

    # Step 2: Get unique domains for WHOIS processing
    unique_domains = list(set([feat["domain"] for feat in feature_rows if feat["domain"]]))
    print(f"\nStep 2: Found {len(unique_domains)} unique domains for WHOIS lookup")

    # Step 3: Process WHOIS in parallel batches
    whois_results = {}
    
    if not SKIP_WHOIS and unique_domains:
        print("Processing WHOIS API calls in parallel...")
        
        # Create batches of domains
        domain_batches = [unique_domains[i:i + BATCH_SIZE] for i in range(0, len(unique_domains), BATCH_SIZE)]
        
        with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            # Submit batches to thread pool
            future_to_batch = {
                executor.submit(process_whois_batch_threaded, batch): batch 
                for batch in domain_batches
            }
            
            # Collect results with progress bar
            for future in tqdm(as_completed(future_to_batch), total=len(domain_batches), desc="WHOIS batches"):
                batch_result = future.result()
                whois_results.update(batch_result)
                time.sleep(0.1)  # Small delay between batches
    else:
        # Skip WHOIS - set all to default value
        whois_results = {domain: 0 for domain in unique_domains}

    # Step 4: Combine results
    print("\nStep 4: Combining results...")
    final_features = []
    
    for features in tqdm(feature_rows, desc="Finalizing features"):
        # Add WHOIS result
        domain = features["domain"]
        features["whois_registered"] = whois_results.get(domain, 0)
        
        # Clean up temporary fields
        del features["domain"]
        del features["hostname"]
        
        final_features.append(features)

    # Step 5: Save results
    print("\nStep 5: Saving results...")
    feature_df = pd.DataFrame(final_features)
    feature_df.to_csv(OUTPUT_FILE, index=False)
    
    print(f"\nFeatures extracted and saved to: {OUTPUT_FILE}")
    print(f"Processed {len(final_features)} URLs")
    print(f"Used {len(whois_results)} unique domains for WHOIS")
    print(f"Cache hits - WHOIS: {len(whois_cache)}, DNS: {len(dns_cache)}")
    
    # Display feature summary
    print(f"\nFeature columns: {list(feature_df.columns)}")
    print(f"Dataset shape: {feature_df.shape}")


Loading dataset...
Loaded 105062 URLs
Analyzing domain distribution...
Total unique domains: 57577
Total URLs: 105062
Average URLs per domain: 1.82

Step 1: Extracting fast features...


Fast features: 100%|██████████| 105062/105062 [17:52:12<00:00,  1.63it/s]     


Extracted fast features for 105062 URLs

Step 2: Found 57684 unique domains for WHOIS lookup
Processing WHOIS API calls in parallel...


WHOIS batches:  53%|█████▎    | 610/1154 [1:17:15<30:42,  3.39s/it]  

WHOIS API error for pornoboard.net (attempt 1): HTTPSConnectionPool(host='www.whoisxmlapi.com', port=443): Max retries exceeded with url: /whoisserver/WhoisService?apiKey=32850693e79c4e1291e441e8f4d09707&domainName=pornoboard.net&outputFormat=JSON (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
WHOIS API error for liberal.hr (attempt 1): HTTPSConnectionPool(host='www.whoisxmlapi.com', port=443): Max retries exceeded with url: /whoisserver/WhoisService?apiKey=32850693e79c4e1291e441e8f4d09707&domainName=liberal.hr&outputFormat=JSON (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
WHOIS API error for goodmart24.ru (attempt 1): HTTPSConnectionPool(host='www.whoisxmlapi.com', port=443): Max retries exceeded with url: /whoisserver/WhoisService?apiKey=32850693e79c4e1291e441e8f4d09707&domainName=goodmart24.ru&outputFormat=JSON (Caused by

...

WHOIS API error for csi-spa.com (attempt 1): HTTPSConnectionPool(host='www.whoisxmlapi.com', port=443): Max retries exceeded with url: /whoisserver/WhoisService?apiKey=32850693e79c4e1291e441e8f4d09707&domainName=csi-spa.com&outputFormat=JSON (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
WHOIS API error for xn--eck3a9bu7culz704abt8c.com (attempt 1): HTTPSConnectionPool(host='www.whoisxmlapi.com', port=443): Max retries exceeded with url: /whoisserver/WhoisService?apiKey=32850693e79c4e1291e441e8f4d09707&domainName=xn--eck3a9bu7culz704abt8c.com&outputFormat=JSON (Caused by SSLError(SSLEOFError(8, '[SSL: UNEXPECTED_EOF_WHILE_READING] EOF occurred in violation of protocol (_ssl.c:1000)')))
WHOIS API error for ea1785.org (attempt 1): HTTPSConnectionPool(host='www.whoisxmlapi.com', port=443): Max retries exceeded with url: /whoisserver/WhoisService?apiKey=32850693e79c4e1291e441e8f4d09707&domainName=ea1785.org&ou

WHOIS batches: 100%|██████████| 1154/1154 [3:48:23<00:00, 11.87s/it]  



Step 4: Combining results...


Finalizing features: 100%|██████████| 105062/105062 [00:00<00:00, 425906.44it/s]



Step 5: Saving results...

Features extracted and saved to: C:\prateek\projects\linkload\backend\ml_models\phishing_detection\data\phishing_dataset_with_features.csv
Processed 105062 URLs
Used 57684 unique domains for WHOIS
Cache hits - WHOIS: 57684, DNS: 78979

Feature columns: ['url_length', 'hostname_length', 'path_length', 'num_dots', 'num_hyphens', 'has_at_symbol', 'has_double_slash', 'has_ip_address', 'has_https', 'count_www', 'count_subdomain', 'first_dir_length', 'has_php_or_html', 'url', 'dns_resolves', 'label', 'whois_registered']
Dataset shape: (105062, 17)
